In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2,os,sys
import numpy as np
from glob import glob
import imutils
import argparse
import shutil
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import glob
import pandas as pd

from tensorflow.keras import losses
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, concatenate, Dropout, BatchNormalization
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.framework import ops

import tensorflow as tf
import matplotlib.pyplot as plt

## for visualizing
import matplotlib.pyplot as plt, numpy as np
from tqdm import tqdm_notebook
import numbers

from tensorflow.python.ops import standard_ops
from tensorflow.python.ops import nn
from tensorflow.python.framework import ops

# import data_generator_dacon as data_generator
# import triplet_loss_dacon as triplet_loss

from sklearn.svm import SVC
import csv

import pickle
import sys
import argparse

In [ ]:
requirements_path = '/content/drive/MyDrive/requirements.txt'

In [ ]:
!pip install -r $requirements_path

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.7/470.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 3

# 이미지 데이터 얼굴만 Crop

In [ ]:
# MTCNN 모델을 추론
def mtcnn_fun(img, min_size, factor, thresholds):
    # MTCNN 모델 파일 경로 지정 및 읽기
    # '/content/drive/MyDrive/mini2_model/resource/mtcnn.pb' 경로에 MTCNN 모델 파일이 있어야 함
    with tf.io.gfile.GFile('/content/drive/MyDrive/mini2_model/resource/mtcnn.pb', 'rb') as f:
        # 그래프 정의 로드
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # 추론을 CPU에서 실행하도록 설정
    with tf.device('/cpu:0'):
        # 세션 생성
        with tf.compat.v1.Session() as sess:
            # 그래프 정의를 사용하여 모델 로드 및 추론 실행
            prob, landmarks, box = tf.import_graph_def(graph_def,
                                                        input_map={
                                                            'input:0': img,  # 입력 이미지
                                                            'min_size:0': min_size,  # 최소 크기
                                                            'thresholds:0': thresholds,  # 임계값
                                                            'factor:0': factor  # 스케일링 요소
                                                        },
                                                        return_elements=[
                                                            'prob:0',  # 얼굴 확률
                                                            'landmarks:0',  # 얼굴 랜드마크
                                                            'box:0'],  # 바운딩 박스
                                                        name='')

            # 추론 결과 출력
            print(box, prob, landmarks)
            # 추론 결과 반환
            return box, prob, landmarks


In [ ]:
#tensorflow 호환성 문제 해결위해 wrap_function 사용
# wrap graph function as a callable function
mtcnn_fun = tf.compat.v1.wrap_function(mtcnn_fun, [
    # 입력 이미지의 형태를 지정합니다. [높이, 너비, 채널] 형태를 가지며, 높이와 너비는 None으로 지정하여 가변적인 크기를 허용합니다.
    # 채널은 RGB 이미지이므로 3으로 지정합니다. 데이터 타입은 tf.float32로 지정합니다.
    tf.TensorSpec(shape=[None, None, 3], dtype=tf.float32),
    # 다음으로는 최소 크기, 스케일링 요소, 임계값에 대한 TensorSpec을 지정합니다.
    # 모두 스칼라 값으로 지정되며, 데이터 타입은 tf.float32로 지정합니다.
    tf.TensorSpec(shape=[], dtype=tf.float32),
    tf.TensorSpec(shape=[], dtype=tf.float32),
    tf.TensorSpec(shape=[3], dtype=tf.float32)
])

Tensor("box:0", dtype=float32, device=/device:CPU:0) Tensor("prob:0", shape=(None,), dtype=float32, device=/device:CPU:0) Tensor("landmarks:0", shape=(None, 10), dtype=float32, device=/device:CPU:0)


In [ ]:
# 원본 데이터 폴더와 같은 데이터 폴더를 생성
def cp_dir(origin_dir,copy_path):
    folder_name_1 = origin_dir.split('/')[-1]
    mk_1 = copy_path +"/"+folder_name_1 + "/"
    if not os.path.exists(mk_1):
        os.makedirs(mk_1)
    return mk_1

In [ ]:
# 이미지에서 face box 좌표에 마진을 주고 자른 얼굴이미지를 리턴
def margine_face(img,box):
    w = box[3] - box[1] # 27
    h = box[2] - box[0] # 20
    w_m = int(h * 0.135)
    h_m = int(w * 0.10)

    x_min = max(box[1] - w_m,0)
    y_min = max(box[0] - h_m,0)
    x_max = min(box[3]+w_m,img.shape[1])
    y_max = min(box[2]+h_m,img.shape[0])

    face_img = img[y_min:y_max,x_min:x_max].copy()

    return face_img

In [ ]:
# 이 코드는 주어진 이미지 디렉토리에서 얼굴을 감지하고, 각 얼굴을 잘라내어 다른 디렉토리에 저장하는 함수입니다. 함수의 각 부분을 주석으로 설명하겠습니다:
suc_cnt = 0
err_path=[]
def write_face(image_dirs,write_path):
    global suc_cnt, err_path
    for file_path in image_dirs:
        img_file_name = file_path.split('/')[-1]
        mk_file = write_path + '/' + img_file_name
        # print(mk_file)

        # read data
        img = cv2.imread(file_path)

        # detect Face
        bbox, scores, landmarks = mtcnn_fun(img, 80, 0.3, [0.98, 0.98, 0.98])  # bbox : face box position
        bbox, scores, landmarks = bbox.numpy(), scores.numpy(), landmarks.numpy()

        total_box = len(bbox)

        if (total_box < 1):
            img = cv2.rotate(img, cv2.ROTATE_180)
            bbox, scores, landmarks = mtcnn_fun(img, 80, 0.3, [0.98, 0.98, 0.98])
            bbox, scores, landmarks = bbox.numpy(), scores.numpy(), landmarks.numpy()
            total_box = len(bbox)

        for box, pts in zip(bbox, landmarks):
            box = tf.cast(box, tf.int32)
            box_np = box.numpy()
            pts = tf.cast(pts, tf.int32)
            f_img = margine_face(img,box_np)
            try:
                f_img = cv2.resize(f_img, (380, 380)) # mean face shape : (155,205,3)
            except:
                continue
            cv2.imwrite(mk_file, f_img)

        if (total_box > 1):
            print("##### total box over ###### ", file_path)
            err_path.append(file_path)

        if (total_box < 1):
            print("##### total box under ###### ",file_path)
            err_path.append(file_path)
        suc_cnt = suc_cnt + total_box

In [ ]:
#이 함수는 데이터셋을 이동하고 일부 이미지를 검증(validation) 데이터셋으로 이동하는 데 사용
def mv_dataset(paths,move_paths,val_str):
    jpg_datas = glob.glob(paths + "*.jpg")
    while (len(jpg_datas) < 1):
        paths += "*/"
        jpg_datas = glob.glob(paths + "*.jpg")
    data_paths = glob.glob(paths + "*.jpg")
    data_paths = np.array(data_paths)
    np.random.shuffle(data_paths)
    i = 2001
    for data in data_paths[:3000]:
        shutil.move(data, move_paths + "image_" + str(i).zfill(5) + ".jpg")
        i += 1

In [ ]:
# 검증 데이터셋 만들기
# 나는 필요 없을
# Create validation dataset
def create_validation_dataset(mv_path):
    fake_paths = mv_path+"/fake/"
    real_paths = mv_path+"/real/"
    move_fake_paths = mv_path+"/validation/fake/samples/"
    if not os.path.exists(move_fake_paths):
        os.makedirs(move_fake_paths)
    move_real_paths = mv_path + "/validation/real/samples/"
    if not os.path.exists(move_real_paths):
        os.makedirs(move_real_paths)

    mv_dataset(fake_paths, move_fake_paths,'fake') # move fake 1000 data
    mv_dataset(real_paths, move_real_paths,'real') # move real 1000 data

In [ ]:
#원본 이미지 데이터셋에서 얼굴을 검출하고, 얼굴이 포함된 이미지만을 선택하여 새로운 데이터셋을 생성하는 역할
total_sample =0
def crop_dataset(origin_paths,mk_paths):
    '''
    원본 데이터 경로 : ./data/CW/20201117/image.jpg
    origin_paths : ./data
    mk_paths : ./make_dataset
    result : ./make_dataset/CW/20201117/image.jpg (얼굴 데이터 )
    '''
    global total_sample

    jpg_datas = glob(origin_paths+"/*.jpg")
    print("total_sample:",total_sample,"face_cnt :",suc_cnt)
    if len(jpg_datas)>0:
        write_face(jpg_datas, mk_paths)
        total_sample += len(jpg_datas)
    else:
        origin_paths += "/*"
        for path in glob(origin_paths):
            mk_dir = cp_dir(path, mk_paths)
            crop_dataset(path+"/",mk_dir)


In [ ]:
# 데이터에서 얼굴을 찾아 자른 얼굴 이미지 리턴
def get_faces(img_paths, input_shape):
    '''
    img_paths : 원본 데이터 경로
    input_shape : 얼굴 이미지의 Shape
    '''
    input_shape = input_shape
    faces = np.zeros((len(img_paths),input_shape[0],input_shape[1],input_shape[2]),dtype=np.float32)
    tmp_dir = "./temp/"
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir)

    i=0
    for path in img_paths:
        img = cv2.imread(path)
        bbox, scores, landmarks = mtcnn_fun(img, 60, 0.7, [0.6, 0.7, 0.8])
        bbox, scores, landmarks = bbox.numpy(), scores.numpy(), landmarks.numpy()
        total_box = len(bbox)

        # not found face box
        if (total_box < 1):
            # 180 rotate image
            img = cv2.rotate(img, cv2.ROTATE_180)
            bbox, scores, landmarks = mtcnn_fun(img, 60, 0.7, [0.6, 0.7, 0.8])
            bbox, scores, landmarks = bbox.numpy(), scores.numpy(), landmarks.numpy()
            total_box = len(bbox)


        for box, pts in zip(bbox, landmarks):
            box = tf.cast(box, tf.int32)
            box_np = box.numpy()
            pts = tf.cast(pts, tf.int32)
            f_img = margine_face(img, box_np)
            f_img = cv2.resize(f_img, (input_shape[0], input_shape[1]))
            cv2.imwrite(tmp_dir+"__temp__.jpg",f_img)
            gfile = tf.io.read_file(tmp_dir+"__temp__.jpg")
            f_img = tf.image.decode_png(gfile)
            os.remove(tmp_dir+"__temp__.jpg")

        faces[i] = f_img
        i+=1
    faces = tf.cast(faces,tf.float32)
    return faces # batchsizextensor


In [ ]:
#메인함수
# 하드코딩된 데이터 경로와 생성할 데이터셋 폴더 경로
data_path_real = "/content/drive/MyDrive/mini2_model/aihub_test/Real_img1"
data_path_fake = "/content/drive/MyDrive/mini2_model/aihub_test/Fake_img1"
create_data_path_real = "/content/drive/MyDrive/mini2_model/cropface/real"
create_data_path_fake = "/content/drive/MyDrive/mini2_model/cropface/fake"
create_data_path = "/content/drive/MyDrive/mini2_model/cropface"

'''
학습용 얼굴 데이터셋 생성
입력:
    데이터셋 경로  ex) /home/dacon/deepfake_1st
    생성할 데이터셋 폴더 경로  ex) /home/dacon/cropface

결과:
    얼굴 데이터셋 생성 (TRAIN 데이터 폴더: fake, real  TEST 데이터 폴더: validation)
'''

# 하드코딩된 값을 사용하여 데이터셋 생성 함수를 호출합니다.
crop_dataset(data_path_real, create_data_path_real)
crop_dataset(data_path_fake, create_data_path_fake)
create_validation_dataset(create_data_path) #train과 test set 나누기

In [ ]:
# 얼굴 인식 불가 이미지 csv 파일에 저장
import csv
csv_file_path = '/content/drive/MyDrive/mini2_model/fake_error_paths.csv'

# CSV 파일에 데이터 쓰기
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for path in err_path:
        writer.writerow([path])

In [ ]:
df = glob.glob("/content/drive/MyDrive/mini2_model/cropface/validation/fake/samples"+"/*")
len_sample = len(df)
df = glob.glob("/content/drive/MyDrive/mini2_model/cropface/real"+"/*")
len_real = len(df)
df = glob.glob("/content/drive/MyDrive/mini2_model/cropface/fake"+"/*")
len_fake = len(df)
print("sample: ", len_sample)
print("real: ", len_real)
print("fake: ", len_fake)
# df_1 = pd.read_csv('/content/drive/MyDrive/mini2_model/fake_error_paths.csv')
# df_1.info()

In [ ]:
# !pip install mtcnn
from mtcnn import MTCNN

detector = MTCNN()

image = cv2.cvtColor(cv2.imread("/content/drive/MyDrive/mini2_model/aihub_test/Real_img1/169762_029_2055.jpg"), cv2.COLOR_BGR2RGB)
result = detector.detect_faces(image)

# Result is an array with all the bounding boxes detected. We know that for 'ivan.jpg' there is only one.
bounding_box = result[0]['box']
keypoints = result[0]['keypoints']

cv2.rectangle(image,
              (bounding_box[0], bounding_box[1]),
              (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
              (0,155,255),
              2)

cv2.circle(image,(keypoints['left_eye']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['right_eye']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['nose']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['mouth_left']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['mouth_right']), 2, (0,155,255), 2)

cv2.imwrite("ivan_drawn.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

print(result)

# data generator

In [ ]:
from tensorflow.keras.utils import Sequence
import tensorflow as tf
# import os
import numpy as np
from imgaug import augmenters as iaa
import random as rnd


import dlib
from imutils import face_utils
import random as rnd
import cv2

In [ ]:
args = {'shape_predictor':'/content/drive/MyDrive/mini2_model/resource/shape_predictor_68_face_landmarks.dat', 'picamera': -1}
detector = dlib.get_frontal_face_detector () #정면 얼굴 감지기 반환
predictor = dlib.shape_predictor (args["shape_predictor"]) #랜드마크 예측

In [ ]:
#입력이미지에서 얼굴을 덮는 작업[장애 요소 추가]
def blind_face(img):
    #     img_ = np.array(img, np.uint8)
    seed = rnd.random ()
    probability = 0.5

    if (seed < probability):
        seed = rnd.random ()
        case = int (seed * 7)
        gray = cv2.cvtColor (img, cv2.COLOR_RGB2GRAY)

        rects = detector (gray, 0)
        if len(rects) > 0 :
            shape = predictor (gray, rects[0])
            shape = face_utils.shape_to_np (shape)
            pts_ = get_poly (shape, case)
            cv2.fillConvexPoly (img, pts_, (0, 0, 0))
            del shape, pts_

        del gray

In [ ]:
# 얼굴의 랜드마크와 선택한 경우를 기반으로 얼굴을 덮는 다각형의 꼭짓점을 계산하는 함수인 get_poly를 정의
def get_poly(shape_, case_):
    input_shape = (380, 380)

    pts = np.array ([shape_[36], shape_[20], shape_[23], shape_[45]], np.int32)

    if case_ == 0:  # both eye
        pt2 = np.array ([shape_[20][0], shape_[36][1] + shape_[36][1] - shape_[20][1]])
        pt1 = np.array ([shape_[23][0], shape_[45][1] + shape_[45][1] - shape_[23][1]])
        pts = np.array ([shape_[36], shape_[20], shape_[23], shape_[45], pt1, pt2], np.int32)
    if case_ == 1:  # upper
        pts = np.array([shape_[0], shape_[17], shape_[26], shape_[16], shape_[14], shape_[34], shape_[32], shape_[2]],
                       np.int32)
    # elif case_ == 1:  # mouth + chic
    #     pts = np.array ([shape_[1], shape_[15], shape_[10], shape_[8], shape_[5]], int32)
    elif case_ == 2:  # left
        pts = np.array ([shape_[0], shape_[17], shape_[21], shape_[27], shape_[8], shape_[5], shape_[3]], np.int32)
    elif case_ == 3:  # right
        pts = np.array ([shape_[16], shape_[26], shape_[22], shape_[27], shape_[8], shape_[11], shape_[14]], np.int32)
    elif case_ == 4:  # nose
        #         pt0 = np.array( [ shape_[27][0], shape_[21][1] ] )
        pt1 = np.array ([shape_[31][0], shape_[28][1]])
        pt2 = np.array ([shape_[35][0], shape_[28][1]])
        pts = np.array ([shape_[27], pt1, shape_[31], shape_[51], shape_[35], pt2], np.int32)
    elif case_ == 5:  # upper
        #         pts = np.array([shape[0], shape_[17], shape_[26], shape_[16], shape_[14], shape_[52], shape_[50], shape_[2]], int32)
        pts = np.array ([shape_[0], shape_[17], shape_[26], shape_[16], shape_[14], shape_[34], shape_[32], shape_[2]],
                        np.int32)
    elif case_ == 6:  # forehead
        # pt1 = np.array ([250, 250])
        pt1 = np.array ([0, shape_[17][1]])
        pt2 = np.array ([input_shape[0] - 1, shape_[26][1]])
        pt3 = np.array ([input_shape[0] - 1, 0])
        pt4 = np.array ([0, 0])
        pts = np.array ([pt1, pt2, pt3, pt4], np.int32)

    return pts


In [ ]:
# 이미지 파일의 경로를 입력으로 받아서 해당 이미지의 레이블을 생성
def get_label(path):
    spl = tf.strings.split (path, "/")
    label = tf.cast (spl[-2] == "fake", tf.uint8)
    label = tf.reduce_sum (label)

    #     class_num = np.argmax (onehot)
    class_num = int (label.numpy ())
    # if(class_num==1):
    #     class_num = np.array([0, 1])
    # else:
    #     class_num = np.array ([1, 0])

    onehot = tf.cast(class_num, tf.uint8)
    return onehot

In [ ]:
    spl = tf.strings.split ("/content/drive/MyDrive/mini2_model/cropface/real/179253_176569_2_0052_342.jpg", "/")
    label = tf.cast (spl[-2] == "fake", tf.uint8)
    label = tf.reduce_sum (label)
    #     class_num = np.argmax (onehot)
    class_num = int (label.numpy ())
    # if(class_num==1):
    #     class_num = np.array([0, 1])
    # else:
    #     class_num = np.array ([1, 0])
    class_num


0

In [ ]:
def load_image_label(path):
    # readimage

    gfile = tf.io.read_file (path)
    image = tf.image.decode_png (gfile)

    del gfile
    # norm
    # image = tf.image.resize (image, (380, 380))
    # image = tf.cast (image, tf.uint8)

    # label
    label = get_label (path)

    return image, label

In [ ]:
class DataGenerator (Sequence):

    def __init__(self, X, y, batch_size, step_per_epoch, input_shape, shuffle=True, augment = True):
        self.X = X
        self.y = y if y is not None else y
        self.batch_size = batch_size
        self.step_per_epoch = step_per_epoch
        self.input_shape = input_shape
        self.shuffle = shuffle
        self.augment = augment
        self.augment_size = int(self.batch_size*0.3)
        self.init_generator ()
        self.on_epoch_end ()

    def _shuffle_sample(self):
        if (self.shuffle == True):
            np.random.shuffle (self.indexes)

    def init_generator(self):
        self.size_of_sample = len (self.X)
        self.indexes = np.zeros (self.size_of_sample, dtype=int)

        for i in range (self.size_of_sample):
            self.indexes[i] = int (i)

        self._shuffle_sample ()

        self.size_of_mini_epoch = self.batch_size * self.step_per_epoch
        self.cnt_mini_epoch = self.size_of_sample // self.size_of_mini_epoch
        self.itor_mini_epoch = 0

    def on_epoch_end(self):

        if (self.itor_mini_epoch < self.cnt_mini_epoch):
            self.mini_indexes = self.indexes[self.itor_mini_epoch * self.size_of_mini_epoch: (self.itor_mini_epoch + 1) * self.size_of_mini_epoch]
            self.itor_mini_epoch = self.itor_mini_epoch + 1
        else:
            self.itor_mini_epoch = 0
            self._shuffle_sample ()
            self.on_epoch_end ()

    def __len__(self):
        return int (self.step_per_epoch)

    #         return int(np.floor(len(self.X) / self.batch_size))

    def __data_generation(self, X_list, y_list):

        # X = np.empty ((self.batch_size, self.input_shape[0], self.input_shape[1], self.input_shape[2]))
        # y = np.empty ((self.batch_size, ), dtype=np.uint8)
        X = np.zeros ((self.batch_size, self.input_shape[0], self.input_shape[1], self.input_shape[2]), dtype=np.float32)
        y = np.zeros ((self.batch_size, ), dtype=int)
        # y = tf.cast(y, tf.uint8)

        if y is not None:
            for i, (path, label) in enumerate(zip(X_list, y_list)):
                img, label_ = load_image_label(path)
                if self.augment and i < self.augment_size:
                    img = augmentor(img)
                # X[i] = tf.cast(img, tf.float32) / 255.0
                X[i] = tf.cast(img, tf.float32)
                y[i] = label_


            return X, y
        else:
            for i, img in enumerate (X_list):
                X[i] = img

            return X

    def __getitem__(self, index):
        indexes = self.mini_indexes[index * self.batch_size: (index + 1) * self.batch_size]
        X_list = [self.X[k] for k in indexes]

        if self.y is not None:
            y_list = [self.y[k] for k in indexes]
            X, y = self.__data_generation (X_list, y_list)
            return X, y
        else:
            y_list = None
            X = self.__data_generation (X_list, y_list)
            return X

In [ ]:
#이미지를 JPEG 형식으로 압축
def image_compression(img, quality):
    encoded_img = tf.image.encode_jpeg(img, format='rgb', quality=quality)
    img = tf.image.decode_jpeg(encoded_img)
    del encoded_img
    return img

In [ ]:
#다양한 방식으로 이미지 변환
def pattern_augment(img):
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5),  # horizontally flip 50% of all images, 수평 뒤집기
            iaa.Flipud(0.2),  # vertically flip 20% of all images, 수직 뒤집
           iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),  # sharpen images, 선명화
           iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),  # emboss images, 입체적인 효과 부여
           ],
           random_order=True
           )
    return seq.augment_image(img)

In [ ]:
noise_pattern_list = ["./noise_pattern.png","./noise_pattern2.png","./noise_pattern3.png"]
def patternBlending(img):
    # 임의의 노이즈 패턴 선택
    noise_pattern = cv2.imread(noise_pattern_list[rnd.randint(0, 2)])
    # 선택된 노이즈 패턴에 패턴 augmentation 적용
    noise_pattern = pattern_augment(noise_pattern)
    # 원본 이미지와 같은 크기로 크기 조정
    noise_pattern = cv2.resize(noise_pattern, (img.shape[1], img.shape[0]))

    # 노이즈 패턴을 원본 이미지에 블렌딩

    w,h = 0,0
    rows, cols, channels = noise_pattern.shape
    m_rows= min(rows+h,img.shape[0]-1)
    m_cols = min(cols + w,img.shape[1]-1)
    roi = img[h:m_rows, w:m_cols,:]
    b_th = 0.2 #노이즈 강도
    img[h:m_rows, w:m_cols,:] = cv2.addWeighted(roi, 1-b_th, noise_pattern[:roi.shape[0],:roi.shape[1]] ,b_th, 0)
    return img

In [ ]:
#다양한 데이터 증강기법 적용
def augmentor(image):
    '''
    image = np.array (image)
    blind_face(image)
    return image
    '''
    # global origin_image,compression_image
    if rnd.random() <= 0.5:
        image = image_compression(image, quality=rnd.randint(10, 30))
    image = np.array(image)
    'Apply data augmentation'
    #if rnd.random() <= 0.3:
        #image = patternBlending(image)
    blind_face (image)
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5),  # horizontally flip 50% of all images
            sometimes(iaa.AdditiveGaussianNoise(scale=(0.05 * 255,0.1 * 255), per_channel=True)),
            sometimes(iaa.GaussianBlur( sigma=(0.0, 1.0) ))
            # sometimes(iaa.UniformColorQuantization(n_colors=32))
        ],
        random_order=True
    )
    return seq.augment_image(image)

# triplet_loss_dacon

In [ ]:
import tensorflow as tf
## required for semi-hard triplet loss:
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes

In [ ]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add (
        math_ops.reduce_sum (math_ops.square (feature), axis=[1], keepdims=True),
        math_ops.reduce_sum (
            math_ops.square (array_ops.transpose (feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul (feature,
                                                     array_ops.transpose (feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum (pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal (pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt (
            pairwise_distances_squared + math_ops.to_float (error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply (
        pairwise_distances, math_ops.to_float (math_ops.logical_not (error_mask)))

    num_data = array_ops.shape (feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like (pairwise_distances) - array_ops.diag (
        array_ops.ones ([num_data]))
    pairwise_distances = math_ops.multiply (pairwise_distances, mask_offdiagonals)
    return pairwise_distances


In [ ]:
def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min (data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max (
        math_ops.multiply (data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

In [ ]:
def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max (data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min (
        math_ops.multiply (data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [ ]:
def triplet_loss_adapted_from_tf(y_true, y_pred):
    # del y_true
    margin = 1.
    # labels = y_pred[:, :1]
    labels = y_true

    labels = tf.cast (labels, dtype='int32')

    # embeddings = y_pred[:, 1:]
    embeddings = y_pred

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:

    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # global batch_size
    batch_size = array_ops.size (labels)  # was 'array_ops.size(labels)'

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance (embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    labels_tile = array_ops.tile(array_ops.reshape(labels, [batch_size, -1]), [1, batch_size])
    # adjacency = math_ops.equal (labels, array_ops.transpose (labels))
    adjacency = math_ops.equal (labels_tile, array_ops.transpose (labels_tile))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not (adjacency)

    # tf.print(adjacency, " adja")



    # Compute the mask.
    pdist_matrix_tile = array_ops.tile (pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and (
        array_ops.tile (adjacency_not, [batch_size, 1]),
        math_ops.greater (
            pdist_matrix_tile, array_ops.reshape (
                array_ops.transpose (pdist_matrix), [-1, 1])))
    # tf.print (mask[:10], 'mask ' )
    mask_final = array_ops.reshape (
        math_ops.greater (
            math_ops.reduce_sum (
                math_ops.cast (mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose (mask_final)

    adjacency_not = math_ops.cast (adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast (mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape (
        masked_minimum (pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose (negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile (
        masked_maximum (pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where (
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add (margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast (
        adjacency, dtype=dtypes.float32) - array_ops.diag (
        array_ops.ones ([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum (mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv (
        math_ops.reduce_sum (
            math_ops.maximum (
                math_ops.multiply (loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')

    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    # tf.print (semi_hard_triplet_loss_distance, 'mask ' )
    return semi_hard_triplet_loss_distance

# 데이터 학습

In [ ]:
server_flag = True #코드 실행 여부
model_load_flag = False #모델 로드 여부
regu_weight = 1e-4 #정규화 가중치, 0.0001에 해당

In [ ]:
# def parse_arguments(argv):
#     parser = argparse.ArgumentParser()
#     parser.add_argument('data_path', type=str,
#                         help='Path of learning data directory ')
#     parser.add_argument ('network_model_path', type=str,
#                          help='output path of network model hdf5 ')
#     parser.add_argument ('svm_model_path', type=str,
#                          help='output path of classifier pickle ')

#     return parser.parse_args(argv)

In [ ]:
def l2_regularizer(scale, scope='l2_regularizer'):
    """Returns a function that can be used to apply L2 regularization to weights.

    Small values of L2 can help prevent overfitting the training data.

    Args:
    scale: A scalar multiplier `Tensor`. 0.0 disables the regularizer.
    scope: An optional scope name.

    Returns:
    A function with signature `l2(weights)` that applies L2 regularization.

    Raises:
    ValueError: If scale is negative or if scale is not a float.
    """
    #정규화 유효성 검사
    if isinstance (scale, numbers.Integral): #정수형인지 확인 - 불가
        raise ValueError ('scale cannot be an integer: %s' % (scale,))
    if isinstance (scale, numbers.Real):#실수형인지 확인
        if scale < 0.:#실수형중 0보다 작으면 불가
            raise ValueError ('Setting a scale less than 0 on a regularizer: %g.' %
                              scale)
    if scale == 0.: #0이면 정규화가 비활성화, 예외는 아님.
        logging.info ('Scale of 0 disables regularizer.')
        return lambda _: None


In [ ]:
model = None
svm_path = None
test_data = None

#콜백 클래스 정의
class On_Epoch_End_Callback (tf.keras.callbacks.Callback):
    def __init__(self, callbacks):
        self.callbacks = callbacks

    def on_epoch_end(self, epoch, logs=None):
        global model, svm_path
        dacon_leaderboard (model, test_data, 50, svm_path) #세번쨰는 에포크 수
        # print ("fake cnt : %d diff : %d epoch : %d " % (fake_, diff_, epoch))
        # for callback in self.callbacks:
        #     callback(self.model)

In [ ]:
def dacon_leaderboard(model, test_data, epoch, svm_path):
    svm_ep = 2
    batch_size = test_data.batch_size

    x_embeddings = np.zeros ((test_data.size_of_sample, 512), np.float32)
    y_embeddings = np.zeros ((test_data.size_of_sample,), int)

    #진행 표시줄
    for it in tqdm_notebook (range (test_data.step_per_epoch)):
        x_train, y_train = test_data.__getitem__ (it)
        x_embeddings[it * batch_size: (it + 1) * batch_size] = model.predict (x_train)
        y_embeddings[it * batch_size: (it + 1) * batch_size] = y_train

    ep_x_embeddings = np.zeros ((test_data.size_of_sample * svm_ep, 512), np.float32)
    ep_y_embeddings = np.zeros ((test_data.size_of_sample * svm_ep,), int)

    #svm 모델을 훈련하고 저장하는 과정
    for it in range (svm_ep):
        ep_x_embeddings[it * test_data.size_of_sample: (it + 1) * test_data.size_of_sample] = x_embeddings
        ep_y_embeddings[it * test_data.size_of_sample: (it + 1) * test_data.size_of_sample] = y_embeddings

    svc_model = SVC (kernel='linear', probability=True) #선형 커널 사용, 클래스 확률 출
    svc_model.fit (ep_x_embeddings, ep_y_embeddings)

    svm_pickle_path = (svm_path + '/ep%d_.pkl' % (epoch)) #훈련된 SVM 모델 저

    del x_embeddings, y_embeddings, ep_x_embeddings, ep_y_embeddings

    with open (svm_pickle_path, 'wb') as outfile:
        pickle.dump ((svc_model), outfile)

In [ ]:
  #참조 코드
  global model, svm_path, test_data
  data_path = "/content/drive/MyDrive/mini2_model/cropface"
  network_model_path = "/content/drive/MyDrive/mini2_model/model/network"
  svm_model_path = "/content/drive/MyDrive/mini2_model/model/svm"

  input_shape = (380, 380, 3)
  if server_flag == True:
      batch_size = 10
  else:
      batch_size = 2

  step_per_epoch = 300

  learning_rate = 0.001
  epochs = 50

  # 380 380 3
  fake_paths = glob.glob (data_path + '/fake/*.jpg')
  real_paths = glob.glob (data_path + '/real/*.jpg')

  val_fake_paths = glob.glob (data_path+'/validation/fake/*/*.jpg')
  val_real_paths = glob.glob (data_path+'/validation/real/*/*.jpg')

  train_paths = fake_paths + real_paths
  test_paths = val_fake_paths + val_real_paths
  # val_steps = len (test_paths) // batch_size #내가 변경
  val_steps = 100
  print (len (train_paths))
  train_data = DataGenerator (train_paths, train_paths, batch_size, step_per_epoch, input_shape, True,True)
  print(train_data)
  test_data = DataGenerator (test_paths, test_paths, batch_size, val_steps, input_shape, True, False) #원래 마지막 false

  # base_network = tf.keras.applications.EfficientNetB4 (input_shape=input_shape, weights=None, include_top=False, pooling='avg')
  input_images = Input (shape=input_shape, name='input_image')  # input layer for images
  base_network = tf.keras.applications.EfficientNetB4 (input_shape=input_shape, input_tensor=input_images,
                                                        weights='imagenet', include_top=False, pooling='avg')

  x = Dropout (0.4) (base_network.output)
  # x = Dense (128, activation='relu') (x)
  x = Dense (512, kernel_regularizer=l2_regularizer (regu_weight), activation=None) (x)
  # x = Dense (512, activation=None) (x)
  # x = BatchNormalization(scale=False) (x)
  outputs = tf.nn.l2_normalize (x, 1, 1e-10)

  # outputs = Dense (1, activation='sigmoid', name='predictions') (x)
  # outputs = Dense (2, activation='softmax', name='predictions') (x)


  svm_path = svm_model_path
  data_path = data_path
  model = Model(inputs=input_images, outputs=outputs)

  if model_load_flag == True:

      load_model_path = '../dacon_ckpt/triplet_effB4_ep11_los0.00_BS28.hdf5'
      load_model = tf.keras.models.load_model (load_model_path, custom_objects={
          'triplet_loss_adapted_from_tf': triplet_loss_adapted_from_tf})

      for layer_target, layer_source in zip (model.layers, load_model.layers):
          weights = layer_source.get_weights ()
          layer_target.set_weights (weights)
          del weights

      del load_model

  # outputs =
  # outputs = base_network ([input_images])

  # opt = Adam (lr=learning_rate)
  # opt = SGD (lr=learning_rate, momentum=0.9)
  opt = SGD (lr=learning_rate)

  # model.compile (loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  # model.compile (loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  model.compile (loss=triplet_loss_adapted_from_tf, optimizer=opt)
  # tf.keras.losses.

  filepath = (network_model_path + "/triplet_effB4_ep{epoch:02d}_BS%d.hdf5" % (batch_size))

  # checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, mode='max', save_best_only=True, period=20)
  # checkpoint = ModelCheckpoint (filepath, monitor='val_loss', verbose=1, mode='min', save_best_only=False, period=5)
  checkpoint2 = ModelCheckpoint (filepath, verbose=1, period=1)
  # callbacks_list = [checkpoint, checkpoint2, On_Epoch_End_Callback(None)]
  callbacks_list = [checkpoint2, On_Epoch_End_Callback (None)]

  # model.fit_generator (
  #     generator=train_data,
  #     validation_data=test_data,
  #     epochs=epochs,
  #     callbacks=callbacks_list,
  # )

  # model.summary()

In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
#수정코드
import glob
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dropout, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow_addons.losses import triplet_semihard_loss
from keras.callbacks import EarlyStopping

# Load data paths
val_steps = 100
fake_paths = glob.glob(data_path + '/fake/*.jpg')
real_paths = glob.glob(data_path + '/real/*.jpg')
val_fake_paths = glob.glob(data_path+'/validation/fake/*/*.jpg')
val_real_paths = glob.glob(data_path+'/validation/real/*/*.jpg')
train_paths = [(path, 1) for path in fake_paths] + [(path, 0) for path in real_paths]
test_paths = [(path, 1) for path in val_fake_paths] + [(path, 0) for path in val_real_paths]

# Create data generators
train_data = DataGenerator(train_paths, train_paths, batch_size, step_per_epoch, input_shape, True, True)
test_data = DataGenerator(test_paths, test_paths, batch_size, val_steps, input_shape, True, False)

# Define base network
input_images = Input(shape=input_shape, name='input_image')
base_network = EfficientNetB4(input_shape=input_shape, input_tensor=input_images,
                              weights='imagenet', include_top=False, pooling='avg')
x = Dropout(0.4)(base_network.output)
x = Dense(512, activation=None)(x)
outputs = Dense (1, activation='sigmoid', name='predictions') (x)
opt = keras.optimizers.Adam(learning_rate=0.0001)

# Compile model
model = Model(inputs=input_images, outputs=outputs)
# opt = SGD(lr=learning_rate)
# opt = Adam (lr=learning_rate)
# model.compile(loss=triplet_semihard_loss, optimizer=opt)

model.compile (loss='binary_crossentropy', optimizer = opt, metrics=['accuracy'])
checkpoint = ModelCheckpoint(
    network_model_path +'/{epoch:03d}-{val_loss:.4f}.hdf5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    mode='auto')

model.summary()

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


71686520/71686520 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_image (InputLayer)    [(None, 380, 380, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 380, 380, 3)          0         ['input_image[0][0]']         
                                                                                                  
 normalization (Normalizati  (None, 380, 380, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     